In [ ]:
%pip install transformers
%pip install datasets
%pip install tokenizers
%pip install torch
%pip install tqdm
%pip install requests
%pip install nltk

In [ ]:
import nltk
import re
import requests
import torch
import pandas as pd

# Ensure NLTK is set up
nltk.download('punkt_tab')


# Data fetching from project Gutenberg

We first download the books from Project Gutenberg and clean them by:
- Removing metadata and irrelevant text.
- Splitting them into meaningful sentences.
- Filtering out incomplete or irrelevant sentences.

This processed data will later be used for **retrieval and question-answering**.

In [ ]:
def is_valid_sentence(sentence):
    """
    Filters out irrelevant sentences:
    - Short sentences (less than 3 characters)
    - Sentences containing mostly Roman numerals or numbers
    - Sentences that look like unstructured lists (multiple commas without a verb)
    """
    sentence = sentence.strip()

    # Ignore very short sentences
    if len(sentence) < 3:
        return False

    # Remove sentences that are mostly Roman numerals or numbers
    if re.fullmatch(r"[IVXLCDM]+\.?", sentence, re.IGNORECASE) or re.fullmatch(r"\d+", sentence):
        return False

    # Filter out lines that seem like lists or metadata (many commas but no verbs)
    if len(re.findall(r",", sentence)) > 3 and not re.search(r"\b(is|was|are|were|has|had|does|do|did|be|will|shall)\b", sentence):
        return False

    return True

In [ ]:
def clean_text(url):
    response = requests.get(url)
    response.raise_for_status()
    print(f"Downloaded the book successfully from {url}!")

    # Extract the main content
    text = response.text
    print("Raw text length:", len(text))

    # Find the book content
    start_match = re.search(r"(?i)(?:CHAPTER I|INTRODUCTION|PREFACE)", text)
    end_match = re.search(r"\*\*\* END OF THE PROJECT GUTENBERG EBOOK", text)

    if start_match and end_match:
        clean_text = text[start_match.start():end_match.start()].strip()
    else:
        print("Warning: Could not locate book boundaries correctly! Using full text as fallback.")
        clean_text = text  # Fallback to using full text

    # Remove unwanted formatting using regex
    clean_text = re.sub(r"\[.*?\]", "", clean_text)  # Remove content inside square brackets
    clean_text = re.sub(r"\s+", " ", clean_text).strip()  # Normalize spaces

    print("Cleaned text length:", len(clean_text))

    # Tokenize into sentences
    sentences = nltk.sent_tokenize(clean_text)

    # Apply filtering
    filtered_sentences = [s for s in sentences if is_valid_sentence(s)]
    print(f"Filtered number of sentences: {len(filtered_sentences)}")

    return filtered_sentences

In [ ]:
# URLs
urls = [
    "https://www.gutenberg.org/cache/epub/56640/pg56640.txt",
    "https://www.gutenberg.org/cache/epub/67813/pg67813.txt",
    "https://www.gutenberg.org/cache/epub/20772/pg20772.txt",
    "https://www.gutenberg.org/cache/epub/40190/pg40190.txt",
    "https://www.gutenberg.org/cache/epub/4924/pg4924.txt",
    "https://www.gutenberg.org/cache/epub/4525/pg4525.txt",
    "https://www.gutenberg.org/cache/epub/40190/pg40190.txt"
]

### Convert loaded data into dataframe

In [ ]:

def create_dataset(urls):
    dataset = []
    book_id = 1  # Assign each book a unique ID

    for url in urls:
        sentences = clean_text(url)  # Get cleaned sentences

        for i, sentence in enumerate(sentences):
            dataset.append({
                "book_id": book_id,
                "text": sentence
            })

        book_id += 1  # Increment book ID for the next book

    # Convert to DataFrame
    df = pd.DataFrame(dataset)

    # Save to CSV
    df.to_csv("ebook_dataset.csv", index=False)
    print("Dataset saved as 'ebook_dataset.csv'")

    return df

In [ ]:
# Create the dataset
df = create_dataset(urls)

In [ ]:
df.head()

In [ ]:
!pip install faiss-cpu

# **DPR**

In [ ]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoder, DPRQuestionEncoderTokenizer, AutoModelForQuestionAnswering, AutoTokenizer
import torch
import faiss

# Load the tokenizer for token-based chunking
device = "cuda" if torch.cuda.is_available() else "cpu"
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

### Chunk Text for Efficient Retrieval

Since DPR works with **fixed-length input sequences**, we split the book text into **overlapping chunks**.

- **Chunk Size:** 512 tokens (for compatibility with DPR).
- **Stride:** 100 tokens (ensuring context overlap).
- **Reason:** Improves retrieval accuracy by preserving sentence context.

This allows the retriever to match queries with the most relevant passages.

In [ ]:
def chunk_text(text, max_tokens=512, stride=100):
    """
    Splits text into overlapping chunks ensuring they respect token limits.
    Uses a sliding window approach to preserve context.
    """
    tokens = ctx_tokenizer(text, truncation=False)["input_ids"]  # Tokenize entire text
    chunks = []

    for i in range(0, len(tokens), max_tokens - stride):  # Overlapping chunks
        chunk = tokens[i : i + max_tokens]  # Extract chunk
        chunk_text = ctx_tokenizer.decode(chunk, skip_special_tokens=True)  # Convert back to text
        chunks.append(chunk_text)

    return chunks

In [ ]:
# Convert Books into Chunks
passages = []
for book_id in df["book_id"].unique():
    book_text = " ".join(df[df["book_id"] == book_id]["text"].tolist())  # Merge all text from a book
    text_chunks = chunk_text(book_text, max_tokens=512, stride=100)

    for chunk in text_chunks:
        passages.append({"book_id": book_id, "text": chunk})

In [ ]:
# Convert to DataFrame for easy use
passage_df = pd.DataFrame(passages)
print("Passages Created:", passage_df.shape)

In [ ]:
# Save for later use
passage_df.to_csv("chunked_passages.csv", index=False)

### Encode Passages Using DPR Context Encoder

To enable fast and efficient retrieval, we encode book passages using the **DPR Context Encoder**.

- **Model Used:** `facebook/dpr-ctx_encoder-single-nq-base`
- **Purpose:** Converts each passage into a dense vector representation.

These embeddings will be stored in a **FAISS index**, allowing fast similarity searches for question answering.

In [ ]:
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base").to(device)

In [ ]:
import numpy as np

def encode_passages(passages, batch_size=8):
    """Encodes passages using DPR Context Encoder in smaller batches to avoid OOM errors."""
    all_embeddings = []

    for i in range(0, len(passages), batch_size):
        batch = passages[i : i + batch_size]  # Take a small batch

        inputs = ctx_tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            embeddings = ctx_encoder(**inputs).pooler_output.cpu().numpy()  # Move to CPU to save GPU memory

        all_embeddings.append(embeddings)
        torch.cuda.empty_cache()  # Free memory after each batch

    return np.vstack(all_embeddings)

In [ ]:
# Encode all book passages
print("Encoding passages for retrieval...")
passage_texts = passage_df["text"].tolist()
passage_embeddings = encode_passages(passage_texts)

### Build FAISS Index for Efficient Search

FAISS (Facebook AI Similarity Search) is used to store and retrieve **DPR passage embeddings**.

- **Index Type:** `IndexFlatIP` (Inner Product similarity for cosine-based retrieval).
- **Function:** Enables efficient **nearest neighbor search** to retrieve the best passage for a query.

This significantly improves the retrieval speed when performing **question-answering or summarization**.

In [ ]:
# Create FAISS Index
print("Building FAISS index...")
faiss_index = faiss.IndexFlatIP(passage_embeddings.shape[1])  # Inner Product similarity
faiss_index.add(passage_embeddings)

In [ ]:
# Load DPR Question Encoder (for query encoding)
q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base").to(device)
q_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

In [ ]:
def encode_query(query):
    """Encodes the query using DPR Question Encoder with memory optimization."""
    inputs = q_tokenizer(
        query,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        embedding = q_encoder(**inputs).pooler_output.cpu().numpy()

    torch.cuda.empty_cache()  # Free GPU memory
    return embedding

### Retrieve Relevant Text from FAISS Index

Once a **query embedding** is generated, we search for the **most relevant passage** using FAISS.

- **Process:**
  1. Compute **similarity scores** between query and stored passage embeddings.
  2. Select the **top-k most relevant passages**.
  3. Return the passage(s) that best answer the question.

This step enables **context-aware question-answering and summarization**.

In [ ]:
def retrieve_relevant_text(query, passage_df, top_k=3):
    """Finds the most relevant passage using FAISS nearest neighbor search."""
    query_embedding = encode_query(query)

    # Search FAISS index
    D, I = faiss_index.search(query_embedding, top_k)  # D = distances, I = indices

    best_passages = [passage_df.iloc[idx]["text"] for idx in I[0]]

    # ✅ Return multiple passages to improve answer extraction
    return best_passages

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, DataCollatorWithPadding
from datasets import Dataset
import random
import json

# Generating Questions Using T5

To improve **Question Answering (QA) performance**, we **automatically generate questions** from book passages using **T5 (Text-To-Text Transfer Transformer)**.

### **Why Use T5 for Question Generation?**
- **Manually creating training questions is slow & inefficient.**
- **T5 can automatically generate relevant questions from text passages.**
- Helps fine-tune **DPR** and **BERT-QA** for better retrieval and answering.


In [ ]:
# Load T5 Question Generation Model
t5_qg_model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)
t5_qg_tokenizer = T5Tokenizer.from_pretrained("t5-small")

### **Load Pre-Trained T5 for Question Generation**
We use a **pre-trained `t5-small` model** to generate questions from text.

- **Model Used:** `t5-small` (or `t5-base` for better accuracy)
- **Tokenizer:** `T5Tokenizer`
- **Task:** `"generate questions: <passage>" → <question>"`

In [ ]:
def generate_questions_t5(passage, num_questions=3):
    """Generates questions from a passage using T5."""
    input_text = "generate questions: " + passage
    inputs = t5_qg_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

    with torch.no_grad():
        question_ids = t5_qg_model.generate(
            inputs.input_ids,
            max_length=50,
            num_beams=5,
            early_stopping=True
        )

    questions = t5_qg_tokenizer.decode(question_ids[0], skip_special_tokens=True)
    return questions.split("\n")[:num_questions]

In [ ]:
# Select More Passages for Question Generation
sample_passages = passage_df["text"].sample(n=100, random_state=42).tolist()  # Pick 100 random passages
generated_questions = []

for passage in sample_passages:
    questions = generate_questions_t5(passage, num_questions=5)  # Generate 5 questions per passage
    generated_questions.extend(questions)

print(f"Generated {len(generated_questions)} training questions!")

# Save the questions
with open("generated_training_questions.txt", "w") as f:
    for q in generated_questions:
        f.write(q + "\n")

print("Questions saved in 'generated_training_questions.txt'")

# Fine-Tuning DPR with Generated Questions

To improve **retrieval accuracy**, we fine-tune the **DPR Question Encoder** using the **auto-generated question-passage pairs**.

### **Why Fine-Tune DPR?**
- **Pre-trained DPR** was trained on general knowledge datasets.
- **Fine-tuning with our dataset** helps it **better retrieve relevant book passages**.
- Improves chatbot response quality by ensuring **accurate passage retrieval**.



In [ ]:
# Load generated questions
with open("generated_training_questions.txt", "r") as f:
    training_questions = [line.strip() for line in f.readlines()]

print(f"Loaded {len(training_questions)} training questions!")

To fine-tune DPR, we need question-passage pairs.

In [ ]:
# Create DPR fine-tuning dataset
dpr_training_data = []
for question in training_questions:
    retrieved_passages = retrieve_relevant_text(question, passage_df, top_k=1)  # Get best matching passage
    best_passage = retrieved_passages[0]  # Best passage

    dpr_training_data.append({
        "question": question,
        "positive_passages": [best_passage]  # Positive passage
    })

In [ ]:
# Save training data as JSON
with open("dpr_training_data.json", "w") as f:
    json.dump(dpr_training_data, f, indent=4)

print(f"DPR Training Data Saved! {len(dpr_training_data)} question-passage pairs")

DPR requires separate tokenization for questions and passages.

In [ ]:
def tokenize_dpr_data(examples):
    """Tokenizes both questions and passages for DPR fine-tuning."""
    # Tokenize questions using the question encoder tokenizer
    question_encodings = q_tokenizer(
        examples["question"], truncation=True, padding="max_length", max_length=512
    )

    # Tokenize passages using the context encoder tokenizer
    passage_encodings = ctx_tokenizer(
        examples["positive_passages"], truncation=True, padding="max_length", max_length=512
    )

    return {
        "input_ids": question_encodings["input_ids"],
        "attention_mask": question_encodings["attention_mask"],
        "input_ids_passage": passage_encodings["input_ids"],
        "attention_mask_passage": passage_encodings["attention_mask"]
    }

In [ ]:
from sklearn.model_selection import train_test_split

# Load dataset and apply tokenization
def load_dpr_training_data():
    """Loads, tokenizes, and splits DPR training data into train & eval sets."""
    with open("dpr_training_data.json", "r") as f:
        data = json.load(f)

    questions = [item["question"] for item in data]
    passages = [item["positive_passages"][0] for item in data]

    # Split data (80% Train, 20% Eval)
    train_qs, eval_qs, train_ps, eval_ps = train_test_split(questions, passages, test_size=0.2, random_state=42)

    # Convert to Hugging Face Dataset format
    train_dataset = Dataset.from_dict({"question": train_qs, "positive_passages": train_ps})
    eval_dataset = Dataset.from_dict({"question": eval_qs, "positive_passages": eval_ps})

    # Apply Tokenization
    train_dataset = train_dataset.map(tokenize_dpr_data, batched=True, remove_columns=["question", "positive_passages"])
    eval_dataset = eval_dataset.map(tokenize_dpr_data, batched=True, remove_columns=["question", "positive_passages"])

    return train_dataset, eval_dataset

# Get tokenized train & eval datasets
dpr_train_dataset, dpr_eval_dataset = load_dpr_training_data()

In [ ]:
from transformers import TrainingArguments, Trainer, AutoModelForSeq2SeqLM
import torch.nn.functional as F

DPR uses contrastive learning, so we define a custom loss function.

In [ ]:
class DPRTrainer(Trainer):
    """Custom Trainer for DPR that computes loss manually."""
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """Computes contrastive loss for DPR Question & Passage encoders."""

        # Extract inputs for DPR
        question_inputs = {
            "input_ids": inputs["input_ids"].to(model.device),
            "attention_mask": inputs["attention_mask"].to(model.device)
        }
        passage_inputs = {
            "input_ids": inputs["input_ids_passage"].to(model.device),
            "attention_mask": inputs["attention_mask_passage"].to(model.device)
        }

        # Get embeddings from DPR encoders
        question_embeds = model(**question_inputs).pooler_output  # Shape: (batch_size, 768)
        passage_embeds = model(**passage_inputs).pooler_output  # Shape: (batch_size, 768)

        # Compute similarity (dot product)
        similarity_scores = torch.matmul(question_embeds, passage_embeds.T)  # Shape: (batch_size, batch_size)

        # Create labels (each question should match its corresponding passage)
        batch_size = question_embeds.size(0)
        labels = torch.arange(batch_size, device=model.device)

        # Compute contrastive loss (CrossEntropy over similarity scores)
        loss = F.cross_entropy(similarity_scores, labels)

        return (loss, {"loss": loss}) if return_outputs else loss

We now train the DPR model using our dataset.

In [ ]:
# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./fine_tuned_dpr",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    report_to="none",
    remove_unused_columns=False  # Prevents Hugging Face from deleting tokenized inputs
)

In [ ]:
trainer = DPRTrainer(
    model=q_encoder,  # Fine-tune the question encoder
    args=training_args,
    train_dataset=dpr_train_dataset,
    eval_dataset=dpr_eval_dataset,
    data_collator=DataCollatorWithPadding(tokenizer=q_tokenizer),
)

print("Starting DPR Fine-Tuning...")
trainer.train()

# Save fine-tuned model
q_encoder.save_pretrained("./fine_tuned_dpr")
print("Fine-Tuned DPR Model Saved!")

# Fine-Tuning BERT-QA & Evaluating DPR

Now that we have **fine-tuned the DPR retriever**, we move on to **evaluating retrieval accuracy** and **fine-tuning BERT-QA** for answer extraction.


In [ ]:
# Load Fine-Tuned Model
q_encoder = DPRQuestionEncoder.from_pretrained("./fine_tuned_dpr").to(device)

def encode_query(query):
    """Encodes the query using the fine-tuned DPR Question Encoder."""
    inputs = q_tokenizer(query, return_tensors="pt").to(device)
    with torch.no_grad():
        embedding = q_encoder(**inputs).pooler_output.cpu().numpy()  # Use fine-tuned model
    return embedding

print("Fine-Tuned DPR Model Loaded!")

We compare retrieval results before & after fine-tuning.

In [ ]:
def test_retrieval_accuracy(queries, passage_df, fine_tuned=True):
    """Test retrieval performance before and after fine-tuning."""
    print("\n=== Testing DPR Retrieval Accuracy ===")

    for query in queries:
        retrieved_passages = retrieve_relevant_text(query, passage_df, top_k=1)
        best_passage = retrieved_passages[0]

        print(f"\n**Query:** {query}")
        print(f"**Retrieved Passage:** {best_passage[:200]}...")  # Show only first 200 chars


In [ ]:
# Test Queries
test_queries = [
    "What is crop rotation?",
    "Explain the benefits of irrigation.",
    "Describe sustainable farming methods.",
    "How does composting improve soil?",
    "What is the best way to maintain soil fertility?",
]

# Test retrieval before fine-tuning (Use old model)
print("\n**Before Fine-Tuning:**")
q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base").to(device)
test_retrieval_accuracy(test_queries, passage_df, fine_tuned=False)

# Test retrieval after fine-tuning (Use fine-tuned model)
print("\n**After Fine-Tuning:**")
q_encoder = DPRQuestionEncoder.from_pretrained("./fine_tuned_dpr").to(device)
test_retrieval_accuracy(test_queries, passage_df, fine_tuned=True)

Once we have high-quality retrieved passages, we need to fine-tune a BERT-QA model to extract accurate answers.

In [ ]:
# Load a moderate-sized BERT-QA model
qa_model_name = "deepset/bert-base-cased-squad2"
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name).to(device)

In [ ]:
def answer_question(question, passages):
    """Extracts the best answer from multiple retrieved passages using BERT-QA."""
    best_answer = None
    best_confidence = 0

    for passage in passages:
        inputs = qa_tokenizer(question, passage, return_tensors="pt", truncation=True, max_length=512).to(device)

        with torch.no_grad():
            outputs = qa_model(**inputs)

        start_probs = torch.nn.functional.softmax(outputs.start_logits, dim=-1)
        end_probs = torch.nn.functional.softmax(outputs.end_logits, dim=-1)

        start_idx = torch.argmax(start_probs)
        end_idx = torch.argmax(end_probs) + 1

        answer_confidence = (start_probs[0][start_idx] + end_probs[0][end_idx - 1]) / 2

        # ✅ Debug: Print confidence scores for each passage
        print(f"\n🔍 QA Debug: Confidence Score = {answer_confidence.item():.4f}")

        if answer_confidence > best_confidence:
            best_confidence = answer_confidence
            best_answer = qa_tokenizer.convert_tokens_to_string(
                qa_tokenizer.convert_ids_to_tokens(inputs.input_ids[0][start_idx:end_idx])
            ).strip()

    # ✅ Handle cases where no confident answer was found
    if not best_answer or best_answer in ["[CLS]", "[SEP]"]:
        return "I couldn't find a clear answer in the retrieved passages."

    return best_answer

We now use DPR to retrieve passages and BERT-QA to extract answers.

In [ ]:
# Generate QA training data
qa_training_data = []
for question in training_questions:
    retrieved_passages = retrieve_relevant_text(question, passage_df, top_k=1)  # Use fine-tuned DPR
    best_passage = retrieved_passages[0]

    answer = answer_question(question, best_passage)  # Extract answer using BERT-QA

    if answer:  # Only keep high-confidence answers
        qa_training_data.append({
            "context": best_passage,
            "question": question,
            "answer": answer,
            "answer_start": best_passage.find(answer)  # Locate answer position in text
        })

In [ ]:
# Save dataset
with open("qa_training_data.json", "w") as f:
    json.dump(qa_training_data, f, indent=4)

print(f"QA Training Data Saved! {len(qa_training_data)} samples created.")

In [ ]:
def tokenize_qa_data(examples):
    """Tokenizes context and question, and aligns answer spans safely."""
    encodings = qa_tokenizer(
        examples["question"],
        examples["context"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

    start_positions = []
    end_positions = []

    for i, answer in enumerate(examples["answer"]):
        answer_text = answer.strip()
        answer_start = examples["answer_start"][i]  # Original character index in the raw context

        # Get tokenized context text
        tokenized_context = qa_tokenizer.convert_ids_to_tokens(encodings["input_ids"][i])

        # Try to find the answer inside the tokenized text
        answer_tokens = qa_tokenizer.tokenize(answer_text)
        answer_length = len(answer_tokens)

        # Try finding the first occurrence of answer tokens inside the tokenized context
        for token_start in range(len(tokenized_context) - answer_length):
            if tokenized_context[token_start: token_start + answer_length] == answer_tokens:
                start_positions.append(token_start)
                end_positions.append(token_start + answer_length - 1)
                break
        else:
            # If answer was not found, assign safe defaults
            start_positions.append(0)
            end_positions.append(0)

    # Ensure valid integer indices
    encodings["start_positions"] = [max(0, pos) for pos in start_positions]
    encodings["end_positions"] = [max(start_positions[i], end_positions[i]) for i in range(len(start_positions))]

    return encodings

In [ ]:
# Load dataset and apply tokenization
def load_qa_training_data():
    """Loads, tokenizes, and prepares QA training data."""
    with open("qa_training_data.json", "r") as f:
        data = json.load(f)

    dataset = Dataset.from_dict({
        "context": [item["context"] for item in data],
        "question": [item["question"] for item in data],
        "answer": [item["answer"] for item in data],
        "answer_start": [item["answer_start"] for item in data],
    })

    # Apply Safe Tokenization
    tokenized_dataset = dataset.map(tokenize_qa_data, batched=True, remove_columns=["context", "question", "answer", "answer_start"])

    return tokenized_dataset

# Get the safely tokenized dataset
qa_train_dataset = load_qa_training_data()


Fine-Tune BERT-QA


In [ ]:
# Define Training Arguments
qa_training_args = TrainingArguments(
    output_dir="./fine_tuned_bert_qa",
    evaluation_strategy="no",
    eval_steps=500,
    save_steps=500,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    report_to="none",
    remove_unused_columns=False  # Prevents Hugging Face from removing necessary columns
)

In [ ]:
# Initialize Trainer
qa_trainer = Trainer(
    model=qa_model,
    args=qa_training_args,
    train_dataset=qa_train_dataset,
)

print("Starting BERT-QA Fine-Tuning...")
qa_trainer.train()

# Save fine-tuned model
qa_model.save_pretrained("./fine_tuned_bert_qa")
print("Fine-Tuned BERT-QA Model Saved!")

# Fine-Tuning T5 for Summarization

To allow the chatbot to generate **concise summaries** of book passages, we fine-tune **T5 (Text-To-Text Transfer Transformer)** for summarization.


In [ ]:
# Load T5 Model & Tokenizer
t5_model_name = "t5-small"  # You can also use "t5-base" for better quality
t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_name)
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_model_name).to(device)

Before fine-tuning, we generate summaries using the pre-trained T5 model.

In [ ]:
def generate_summary(passage):
    """Generates a summary from a passage using T5."""
    input_text = "summarize: " + passage
    inputs = t5_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

    with torch.no_grad():
        summary_ids = t5_model.generate(
            inputs.input_ids,
            max_length=150,
            num_beams=5,
            length_penalty=0.7,  # Encourage concise summaries
            early_stopping=True
        )

    return t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

We sample 100 random passages from the books and generate summaries.

In [ ]:
# Generate Training Data for Summarization
t5_training_data = []
sample_passages = passage_df["text"].sample(n=500, random_state=42).tolist()  # Pick 100 random passages

for passage in sample_passages:
    summary = generate_summary(passage)  # Generate summary with pretrained T5
    t5_training_data.append({
        "text": passage,
        "summary": summary
    })

In [ ]:
# Save dataset
with open("t5_training_data.json", "w") as f:
    json.dump(t5_training_data, f, indent=4)

print(f"Summarization Training Data Saved! {len(t5_training_data)} samples created.")

We load the dataset and apply tokenization.

In [ ]:
def tokenize_t5_data(examples):
    """Tokenizes input text and summary for T5 fine-tuning."""
    model_inputs = t5_tokenizer(
        examples["text"],
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    labels = t5_tokenizer(
        examples["summary"],
        max_length=150,
        truncation=True,
        padding="max_length"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:

# Load dataset and apply tokenization
def load_t5_training_data():
    """Loads and tokenizes T5 training data."""
    with open("t5_training_data.json", "r") as f:
        data = json.load(f)

    dataset = Dataset.from_dict({
        "text": [item["text"] for item in data],
        "summary": [item["summary"] for item in data]
    })

    # Apply Tokenization
    tokenized_dataset = dataset.map(tokenize_t5_data, batched=True, remove_columns=["text", "summary"])

    return tokenized_dataset

# Get tokenized training dataset
t5_train_dataset = load_t5_training_data()

Once the dataset is ready, we train T5 on our book passages.

In [ ]:
# Define Training Arguments
t5_training_args = TrainingArguments(
    output_dir="./fine_tuned_t5",
    evaluation_strategy="no",  # No validation dataset
    save_steps=500,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    report_to="none",
    remove_unused_columns=False  # Prevents Hugging Face from deleting tokenized inputs
)

# Fine-Tune T5
t5_trainer = Trainer(
    model=t5_model,
    args=t5_training_args,
    train_dataset=t5_train_dataset,
)

In [ ]:
print("Starting T5 Fine-Tuning...")
t5_trainer.train()

# Save the fine-tuned model
t5_model.save_pretrained("./fine_tuned_t5")
print("Fine-Tuned T5 Model Saved!")

### Building the Conversational Chatbot

Now that we have **fine-tuned DPR, BERT-QA, and T5**, we integrate them into a **Gradio-based chatbot** that can:
-  **Summarize agricultural topics** using T5.
-  **Answer questions** using BERT-QA.
-  **Retrieve relevant book passages** using DPR.


In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

# Load Fine-Tuned Models
q_encoder = DPRQuestionEncoder.from_pretrained("./fine_tuned_dpr").to(device)
qa_model = AutoModelForQuestionAnswering.from_pretrained("./fine_tuned_bert_qa").to(device)
t5_model = AutoModelForSeq2SeqLM.from_pretrained("./fine_tuned_t5").to(device)

The chatbot detects user intent and selects the appropriate model.

In [ ]:
def chatbot_response(user_input):
    """Processes user input and returns either a QA answer or a summary."""

    # Detect user intent
    if any(kw in user_input.lower() for kw in ["tell me about", "describe", "summarize", "explain"]):
        task = "summarization"
    elif user_input.strip().endswith("?"):
        task = "qa"
    else:
        return "I'm not sure how to answer that. Try asking a question or requesting a summary."

    # Retrieve the most relevant passage using fine-tuned DPR
    retrieved_passages = retrieve_relevant_text(user_input, passage_df, top_k=1)
    best_passage = retrieved_passages[0]

    # Execute the correct task
    if task == "qa":
        response = answer_question(user_input, best_passage)
        return f"**Answer:** {response}\n\n**Source:** {best_passage}"
    elif task == "summarization":
        response = generate_summary(best_passage)
        return f"**Summary:** {response}\n\n**Source:** {best_passage}"

We use Gradio to create an interactive chatbot interface.

In [ ]:
# Launch the chatbot
gr.Interface(
    fn=chatbot_response,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question or request a summary..."),
    outputs="text",
    title="Conversational Agriculture Chatbot",
    description="Ask about agricultural topics. Try 'What is crop rotation?' or 'Tell me about soil fertility.'",
    live=True
).launch()

We define a set of test cases to verify chatbot behavior.

In [ ]:
test_cases = [
    {"input": "What is crop rotation?", "expected": "QA"},
    {"input": "Tell me about soil fertility.", "expected": "Summarization"},
    {"input": "Explain the benefits of irrigation.", "expected": "Summarization"},
    {"input": "How does composting help soil?", "expected": "QA"},
    {"input": "Describe sustainable farming.", "expected": "Summarization"},
    {"input": "Who wrote this book?", "expected": "QA"},
    {"input": "What is the capital of France?", "expected": "QA"},
    {"input": "Summarize this book.", "expected": "Summarization"},
    {"input": "How can I improve my soil quality?", "expected": "QA"},
]

In [ ]:
def test_chatbot():
    """Runs predefined test cases to evaluate chatbot performance."""
    results = []

    print("\n=== Running Chatbot Tests ===")

    for i, case in enumerate(test_cases):
        print(f"\n**Test Case {i+1}: {case['input']}**")

        response = chatbot_response(case["input"])

        # Detect output type
        if "**Answer:**" in response:
            detected = "QA"
        elif "**Summary:**" in response:
            detected = "Summarization"
        else:
            detected = "Unknown"

        # Store results
        result = {
            "Test Case": i + 1,
            "Input": case["input"],
            "Expected": case["expected"],
            "Detected": detected,
            "Pass": detected == case["expected"],
            "Response": response[:200]  # Show only first 200 chars
        }
        results.append(result)

        print(f"  - Expected: {case['expected']}")
        print(f"  - Detected: {detected}")
        print(f"  - Pass: {result['Pass']}")
        print(f"  - Response Preview: {result['Response']}...")

    return results

In [ ]:
# Run the test cases
test_results = test_chatbot()

# Reflection on the Chatbot's Performance

This project aims to implement a multi-component NLP chatbot capable of retrieving passages, summarizing content, and answering questions using fine-tuned DPR, BERT-QA, and T5. However, upon further evaluation, a critical issue was identified—the chatbot consistently returned [CLS] as the answer to all QA queries.

### Analysis of the [CLS] Response Issue
The chatbot was expected to extract meaningful answers from book passages using BERT-QA. Instead, it returned [CLS] for every response, indicating that the model failed to extract proper answer spans.

### Potential Causes & Fixes
1. Incorrect Token Alignment in BERT-QA
2. Low Confidence in Answer Extraction
3. Poor Passage Retrieval from DPR

# Final thoughts

The chatbot successfully classifies queries but fails to provide valid QA responses. Fixing passage retrieval and token alignment should significantly improve performance. Once these adjustments are made, the chatbot will be better equipped to answer questions reliably rather than defaulting to [CLS]